In [1]:
using Flux

In [2]:
# Define the maximum length of the input text strings
max_len = 120

# Define the number of unique characters in the text
n_chars = 128

# Define the number of neurons in the encoder and decoder layers
n_encoder = 128
n_decoder = 128

# Define the noise probability for the denoising process
noise_prob = 0.5

0.5

In [3]:
function softmax(x::AbstractVector)
    # Compute the exponential of each element in the input vector
    exp_x = exp.(x)

    # Normalize the exponential values by dividing by the sum of the exponentials
    norm_exp_x = exp_x ./ sum(exp_x)

    return norm_exp_x
end


function pad_array(arr::Array, final_length::Int, padval::Char, direction::Symbol)
    # Calculate the number of padding elements needed
    n_pad = final_length - length(arr)

    # Initialize the padded array
    padded_arr = copy(arr)

    # Add padding elements to the array
    if direction == :left
        for i in 1:n_pad
            pushfirst!(padded_arr, padval)
        end
    elseif direction == :right
        for i in 1:n_pad
            push!(padded_arr, padval)
        end
    end

    return padded_arr
end

pad_array (generic function with 1 method)

In [4]:
# Define the autoencoder model
function autoencoder(max_len, n_chars, n_encoder, n_decoder, noise_prob)	
	
    # Define the encoder and decoder layers
    encoder = Chain(
        LSTM(n_chars, n_encoder),
        Dense(n_encoder, n_encoder)
    )

    decoder = Chain(
        Dense(n_encoder, n_decoder),
        LSTM(n_decoder, n_chars),
        softmax
    )

    # Define the autoencoder model by stacking the encoder and decoder layers
    model = Chain(encoder, decoder)

    # Define the loss function and optimization algorithm
    loss(x, y) = crossentropy(model(x), y)
    opt = ADAM()

    # Define the function for training the autoencoder
    ps = Flux.params(model)
    function train_autoencoder(data, epochs)
		for epoch in 1:2
		   Flux.train!(loss,ps, zip(data), opt)
		end				
    end

    # Define the function for denoising the input text
    function denoise_text(text)
        # Convert the input text to a one-hot encoded tensor
        text_oh = onehotbatch(text, 1:n_chars)

        # Add noise to the input tensor with probability `noise_prob`
        text_noisy = map(x -> x .* (rand(size(x)) .> noise_prob), text_oh)

        # Denoise the input text by passing it through the autoencoder model
        text_denoised = model(text_noisy)

        # Convert the denoised tensor back to text
        return String(argmax(text_denoised, dims=2))
    end

    return model, train_autoencoder, denoise_text
end

autoencoder (generic function with 1 method)

In [5]:
# Define a test text string
test_text = "This is a test text string to be denoised by the autoencoder."

# Pad the test text string to the maximum length
test_arr = collect(test_text)
test_text_padded = pad_array(test_arr,n_chars,'0',:right)

128-element Vector{Char}:
 'T': ASCII/Unicode U+0054 (category Lu: Letter, uppercase)
 'h': ASCII/Unicode U+0068 (category Ll: Letter, lowercase)
 'i': ASCII/Unicode U+0069 (category Ll: Letter, lowercase)
 's': ASCII/Unicode U+0073 (category Ll: Letter, lowercase)
 ' ': ASCII/Unicode U+0020 (category Zs: Separator, space)
 'i': ASCII/Unicode U+0069 (category Ll: Letter, lowercase)
 's': ASCII/Unicode U+0073 (category Ll: Letter, lowercase)
 ' ': ASCII/Unicode U+0020 (category Zs: Separator, space)
 'a': ASCII/Unicode U+0061 (category Ll: Letter, lowercase)
 ' ': ASCII/Unicode U+0020 (category Zs: Separator, space)
 't': ASCII/Unicode U+0074 (category Ll: Letter, lowercase)
 'e': ASCII/Unicode U+0065 (category Ll: Letter, lowercase)
 's': ASCII/Unicode U+0073 (category Ll: Letter, lowercase)
 ⋮
 '0': ASCII/Unicode U+0030 (category Nd: Number, decimal digit)
 '0': ASCII/Unicode U+0030 (category Nd: Number, decimal digit)
 '0': ASCII/Unicode U+0030 (category Nd: Number, decimal digit)
 '

In [7]:
# Initialize the autoencoder model
model, train_autoencoder, denoise_text = 
	autoencoder(max_len, n_chars, n_encoder, n_decoder, noise_prob)

(Chain(Chain(Recur(LSTMCell(128 => 128)), Dense(128 => 128)), Chain(Dense(128 => 128), Recur(LSTMCell(128 => 128)), softmax)), train_autoencoder, denoise_text)

In [8]:
# Train the autoencoder model on the test text
train_autoencoder(test_text_padded, 1)

LoadError: MethodError: no method matching (::var"#loss#2"{Chain{Tuple{Chain{Tuple{Flux.Recur{Flux.LSTMCell{Matrix{Float32}, Matrix{Float32}, Vector{Float32}, Tuple{Matrix{Float32}, Matrix{Float32}}}, Tuple{Matrix{Float32}, Matrix{Float32}}}, Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}}}, Chain{Tuple{Dense{typeof(identity), Matrix{Float32}, Vector{Float32}}, Flux.Recur{Flux.LSTMCell{Matrix{Float32}, Matrix{Float32}, Vector{Float32}, Tuple{Matrix{Float32}, Matrix{Float32}}}, Tuple{Matrix{Float32}, Matrix{Float32}}}, typeof(softmax)}}}}})(::Char)
[0mClosest candidates are:
[0m  (::var"#loss#2")(::Any, [91m::Any[39m) at In[4]:20